In [2]:
!pip install boto3 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.4 MB/s eta 0:00:00


In [20]:
import boto3
import json
import uuid
from google.colab import userdata
from datetime import datetime
aws_access_key_id = userdata.get('aws_access_key_id')
aws_secret_key = userdata.get('aws_secret_key')

In [15]:
# Initialize Bedrock client
bedrock_agent_client = boto3.client(
    "bedrock-agent",
    region_name="us-east-2",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_key,
    )

# Define the agent name and description
agent_name = "CustomerSupportAgent"
agent_description = "A chatbot for assisting customers with different tasks."

In [5]:
agent_instruction = '''
You are a friendly and helpful customer service associate working at an outdoor cloth e-commerse company called 'Backcountry'. Your name is Dennis.

Your task is to help customers with inquries on products. Briefly introduce yourself and your task at the beginning of the conversation and ask for customer's information [Full Name, Gender, Age, Activities] to better assist them. After customer reply, use the tool provided to store the info to S3 bucket. Then ask 'Thanks for providing the info! How can I help you today?'. If customer ignore the message or choose to not share the info, do not ask again.

When customer ask for recommendation, search the knowledge base try to give 2 recommendation with details in bullet points for customer to choose. If you cannot find the product, apologies to the customer and state that we don't currently sell this type of product.

'''



In [7]:
# Create the agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    description=agent_description,
    instruction=agent_instruction,
    foundationModel="anthropic.claude-3-7-sonnet-20250219-v1:0",
    idleSessionTTLInSeconds=300  # Set session timeout (5 minutes)
)

# Print the response
print("Agent Created:", response)

Agent Created: {'ResponseMetadata': {'RequestId': '7e01c660-c38a-41cf-b368-dbd075b5b0dd', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Thu, 03 Apr 2025 05:47:15 GMT', 'content-type': 'application/json', 'content-length': '1379', 'connection': 'keep-alive', 'x-amzn-requestid': '7e01c660-c38a-41cf-b368-dbd075b5b0dd', 'x-amz-apigw-id': 'IboTkG0qCYcEYhQ=', 'x-amzn-trace-id': 'Root=1-67ee20e3-1a0e253f4555a23c60e40d4e'}, 'RetryAttempts': 0}, 'agent': {'agentArn': 'arn:aws:bedrock:us-east-2:324037274971:agent/LPHJ94LMYT', 'agentCollaboration': 'DISABLED', 'agentId': 'LPHJ94LMYT', 'agentName': 'CustomerSupportAgent', 'agentStatus': 'CREATING', 'createdAt': datetime.datetime(2025, 4, 3, 5, 47, 15, 308502, tzinfo=tzlocal()), 'description': 'A chatbot for assisting customers with different tasks.', 'foundationModel': 'anthropic.claude-3-7-sonnet-20250219-v1:0', 'idleSessionTTLInSeconds': 300, 'instruction': "\nYou are a friendly and helpful customer service associate working at an outdoor clot

#Testing Agent prepared using Console

## Simple Invoke

In [27]:
bedrock_run_agent_client = boto3.client(
    "bedrock-agent-runtime",
    region_name="us-east-2",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_key,
    )

prompt = 'I do not want to provide personal info, directly recommend me a jacket that is warm and comfortable.'

session_id = uuid.uuid4().hex

response = bedrock_run_agent_client.invoke_agent(
            agentId='7AKAZ4EHGY',
            agentAliasId='GZQHDO1MSS',
            sessionId=session_id,
            inputText=prompt,
        )

completion = ""

for event in response.get("completion"):
    chunk = event["chunk"]
    completion += chunk["bytes"].decode()

print(completion)

Here are two warm and comfortable jacket recommendations:

1. Nano Puff Vest by Patagonia
- Light and packable vest for cold climates
- Synthetic insulation that traps heat even when wet
- Water-repellent finish
- Price range: $132.30 - $189.00
- Available in 8 colors
- Great for alpine, skiing, and climbing activities

2. Antora Jacket by The North Face
- Waterproof and wind-resistant
- Recycled polyester fabric with abrasion resistance
- Three-piece hood to trap heat and fight off rain
- Price range: $66.00 - $110.00
- Available in 11 colors
- Suitable for hiking, camping, and casual wear

Would you like more details about either of these jackets?


# Function to invoke
## Memory not supported yet.

In [23]:
def invoke_bedrock_agent_with_streaming(agentId, agentAliasId, prompt, region_name):
    """
    Invoke an AWS Bedrock agent with streaming response.
    """
    # Create a Bedrock Agent Runtime client
    bedrock_agent_runtime = boto3.client(
        service_name='bedrock-agent-runtime',
        region_name=region_name,
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_key
    )

    session_id = uuid.uuid4().hex

    # Invoke the agent with streaming enabled
    response = bedrock_agent_runtime.invoke_agent(
        agentId=agentId,
        agentAliasId=agentAliasId,
        sessionId=session_id,
        inputText=prompt,
        enableTrace=False  # Skip trace for now
    )

    # Process the streaming response
    full_response = ""
    print("Agent response:")
    for event in response['completion']:
        if 'chunk' in event:
            chunk = event['chunk']
            if 'bytes' in chunk:
                # Decode and process text chunk
                text_chunk = chunk['bytes'].decode('utf-8')
                full_response += text_chunk
                print(text_chunk, end='', flush=True)  # Stream to console

    print("\n")  # Add newline after response
    return full_response


if __name__ == "__main__":

    # Agent prepared on AWS console
    agentId = "7AKAZ4EHGY"
    agentAliasId = "GZQHDO1MSS"
    region_name = "us-east-2"


    # Simple interactive loop
    print("Chat with Bedrock Agent (type 'exit' to quit)")
    while True:
        user_input = input("\nYou: ")

        if user_input.lower() == 'exit':
            break

        # Invoke the agent and stream the response
        invoke_bedrock_agent_with_streaming(
            agentId=agentId,
            agentAliasId=agentAliasId,
            prompt=user_input,
            region_name=region_name
        )

Chat with Bedrock Agent (type 'exit' to quit)

You: Hi, can you recommend a water proof jacket at price about 200 dollars?
Agent response:
I have two waterproof jacket recommendations for you:

1. Patagonia Torrentshell 3L Jacket - Men's
- Price: $179.00
- Features:
  * Rugged, lightweight shell with waterproof protection
  * Breathable 3-layer membrane
  * Underarm vents for heat management
  * Adjustable hood
  * Zips into its own pocket for easy storage
  * Fully sealed seams
  * Suitable for hiking and casual activities

2. The North Face Antora Jacket - Men's
- Price: $66.00 - $110.00
- Features:
  * Waterproof and wind-resistant
  * Recycled polyester face fabric
  * Three-piece adjustable hood
  * Zippered hand pockets
  * Fully sealed seams
  * Great for hiking, camping, and casual wear

Would you like more details about either of these jackets?


You: Sure, tell me about the material
Agent response:
Hi there! I'm Danny, a customer service associate at Mountain Hardwear. Before